# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
# Import library yang dibutuhkan
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
import joblib

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [2]:
# Memuat dataset hasil clustering
file_path = '/content/Dataset_clustering.csv'  # Ganti path sesuai lokasi file di Colab
data = pd.read_csv(file_path)
print("Dataset berhasil dimuat!")
print(data.head())

Dataset berhasil dimuat!
                                   nama_variabel nama_variabel_turunan  \
0  Tingkat Pengangguran Terbuka Menurut Provinsi             Tidak Ada   
1  Tingkat Pengangguran Terbuka Menurut Provinsi             Tidak Ada   
2  Tingkat Pengangguran Terbuka Menurut Provinsi             Tidak Ada   
3  Tingkat Pengangguran Terbuka Menurut Provinsi             Tidak Ada   
4  Tingkat Pengangguran Terbuka Menurut Provinsi             Tidak Ada   

   nama_tahun nama_turunan_tahun  data_content nama_item_vertical_variabel  \
0        2023           Februari          5.75                        ACEH   
1        2023           Februari          5.24              SUMATERA UTARA   
2        2023           Februari          5.90              SUMATERA BARAT   
3        2023           Februari          4.25                        RIAU   
4        2023           Februari          4.50                       JAMBI   

   data_content_scaled content_category  Cluster  
0         

# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [3]:
# Preprocessing Data
# Encode kolom kategorikal
categorical_columns = ['nama_variabel', 'nama_variabel_turunan', 'nama_turunan_tahun',
                       'nama_item_vertical_variabel', 'content_category']
encoder = LabelEncoder()
for col in categorical_columns:
    data[col] = encoder.fit_transform(data[col])

# Memilih fitur dan label
features = ['data_content', 'data_content_scaled', 'nama_tahun'] + categorical_columns
X = data[features]
y = data['Cluster']

# Standarisasi data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Membagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Tampilkan bentuk data
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

Shape of X_train: (1148, 8)
Shape of X_test: (287, 8)


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [4]:
# Model 1: Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

# Model 2: Logistic Regression
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)

Tulis narasi atau penjelasan algoritma yang Anda gunakan.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [5]:
# Evaluasi Random Forest
print("Random Forest - Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Random Forest - F1 Score:", f1_score(y_test, y_pred_rf, average='weighted'))

# Evaluasi Logistic Regression
print("Logistic Regression - Accuracy:", accuracy_score(y_test, y_pred_lr))
print("Logistic Regression - F1 Score:", f1_score(y_test, y_pred_lr, average='weighted'))

Random Forest - Accuracy: 1.0
Random Forest - F1 Score: 1.0
Logistic Regression - Accuracy: 0.9825783972125436
Logistic Regression - F1 Score: 0.9826478109690765


Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [6]:
# Hyperparameter tuning untuk Random Forest
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}
grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=3, scoring='f1_weighted')
grid_rf.fit(X_train, y_train)
print("Best parameters for Random Forest:", grid_rf.best_params_)

Best parameters for Random Forest: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}


## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [7]:
# Menggunakan model terbaik dari GridSearchCV
best_rf_model = grid_rf.best_estimator_
y_pred_best_rf = best_rf_model.predict(X_test)

# Evaluasi ulang model Random Forest
print("Tuned Random Forest - Accuracy:", accuracy_score(y_test, y_pred_best_rf))
print("Tuned Random Forest - F1 Score:", f1_score(y_test, y_pred_best_rf, average='weighted'))

Tuned Random Forest - Accuracy: 1.0
Tuned Random Forest - F1 Score: 1.0


## **e. Analisis Hasil Evaluasi Model Klasifikasi**

In [8]:
# Confusion Matrix dan Classification Report untuk kedua model
print("\nConfusion Matrix - Random Forest")
print(confusion_matrix(y_test, y_pred_rf))
print("\nConfusion Matrix - Logistic Regression")
print(confusion_matrix(y_test, y_pred_lr))

# Analisis hasil dengan Classification Report
print("\nClassification Report - Random Forest")
print(classification_report(y_test, y_pred_rf))

print("\nClassification Report - Logistic Regression")
print(classification_report(y_test, y_pred_lr))



Confusion Matrix - Random Forest
[[ 56   0   0]
 [  0 115   0]
 [  0   0 116]]

Confusion Matrix - Logistic Regression
[[ 54   2   0]
 [  1 114   0]
 [  2   0 114]]

Classification Report - Random Forest
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        56
           1       1.00      1.00      1.00       115
           2       1.00      1.00      1.00       116

    accuracy                           1.00       287
   macro avg       1.00      1.00      1.00       287
weighted avg       1.00      1.00      1.00       287


Classification Report - Logistic Regression
              precision    recall  f1-score   support

           0       0.95      0.96      0.96        56
           1       0.98      0.99      0.99       115
           2       1.00      0.98      0.99       116

    accuracy                           0.98       287
   macro avg       0.98      0.98      0.98       287
weighted avg       0.98      0.98      0.98

Berikut adalah **rekomendasi** tahapannya.
1. Perbandingan Hasil Evaluasi Sebelum dan Sesudah Tuning

Sebelum dilakukan tuning, model Random Forest sudah menunjukkan performa yang sangat baik dengan akurasi dan F1-Score mencapai 100%. Setelah tuning, hasil yang diperoleh tetap sama, sehingga tidak ada peningkatan yang signifikan. Sementara itu, Logistic Regression juga menunjukkan kinerja yang tinggi dengan akurasi dan F1-Score sekitar 98.26%.

2. Identifikasi Kelemahan Model
	* Precision atau Recall Rendah pada Kelas Tertentu:
Tidak ditemukan kendala terkait precision atau recall yang rendah. Model Random Forest memberikan hasil sempurna untuk semua kelas, sedangkan Logistic Regression memiliki sedikit kesalahan yang tidak terlalu berdampak besar.
	* Overfitting atau Underfitting:
Random Forest kemungkinan mengalami overfitting, terutama jika dataset yang digunakan relatif kecil, karena model memberikan hasil yang sempurna. Sebaliknya, Logistic Regression tampak lebih stabil dan mampu melakukan generalisasi dengan baik.

3. Rekomendasi untuk Langkah Selanjutnya
	* Untuk mengurangi potensi overfitting pada Random Forest, dapat dilakukan pembatasan kedalaman pohon atau menambahkan lebih banyak data.
	*	Logistic Regression dapat ditingkatkan dengan menambahkan fitur tambahan atau mencoba algoritma lain seperti Gradient Boosting jika diperlukan.
	*	Jika dataset yang digunakan masih terbatas, disarankan untuk mengumpulkan lebih banyak data guna meningkatkan kemampuan model dalam melakukan generalisasi.

